# Optimization of Noise Generator

We want to find the right Hyperparameters for the Noise Generator.
For the Optimization, we will use the `Optuna` Framework

The correct Hyperparameters for the Noise will be found for:
- `GeFeU`
- `GEMU`

In [8]:
import optuna
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
import random

import os
import sys

# Add the parent directory to sys.path
sys.path.append(os.path.join('..', 'src'))

import gefeu
import gemu
import mlp_dataclass
import metrics
from helper import load_models_dict

___
## GeFeU

In [9]:
import logging
import sys
import pickle
import optuna
import os

# Add stream handler of stdout to show the messages
optuna.logging.get_logger("optuna").addHandler(logging.StreamHandler(sys.stdout))
study1_name = "GenOptiGeFeU"  # Unique identifier of the study.
storage1_name = "sqlite:///{}.db".format(study1_name)

if os.path.exists("sampler_gefeu.pkl"):
    restored1_sampler = pickle.load(open("sampler_gefeu.pkl", "rb"))
    study_gefeu = optuna.create_study(study_name=study1_name, storage=storage1_name, load_if_exists=True, sampler=restored1_sampler)
else:
    study_gefeu = optuna.create_study(study_name=study1_name, storage=storage1_name, load_if_exists=True)

[I 2025-01-09 09:34:07,010] Using an existing study with name 'GenOptiGeFeU' instead of creating a new one.


Using an existing study with name 'GenOptiGeFeU' instead of creating a new one.
Using an existing study with name 'GenOptiGeFeU' instead of creating a new one.


In [10]:
def objective(trial):

    opt_Epochs = trial.suggest_int('opt_Epochs', 1, 10)
    opt_Learning_Rate = trial.suggest_float('opt_Learning_Rate', 0.01, 0.3)
    opt_Batch_Size = trial.suggest_int('opt_Batch_Size', 32, 256)
    opt_N2R_Ratio = trial.suggest_float('opt_N2R_Ratio', 0.01, 20)
    opt_Regularization_term = trial.suggest_float('opt_Regularization_term', 0.01, 0.3)
    opt_Noise_Dim = trial.suggest_int('opt_Noise_Dim', 1, 512)
    opt_Impair_LR = trial.suggest_float('opt_Impair_LR', 0.01, 0.3)
    opt_Repair_LR = trial.suggest_float('opt_Repair_LR', 0.01, 0.3)

    l1 = trial.suggest_int('l1', 32, 1024)
    l2 = trial.suggest_int('l2', 32, 1024)
    l3 = trial.suggest_int('l3', 32, 1024)
    l4 = trial.suggest_int('l4', 32, 1024)
    l5 = trial.suggest_int('l5', 32, 1024)
    l6 = trial.suggest_int('l6', 32, 1024)
    l7 = trial.suggest_int('l7', 32, 1024)
    l8 = trial.suggest_int('l8', 32, 1024)
    l9 = trial.suggest_int('l9', 32, 1024)
    n_layers = trial.suggest_int('n_layers', 1, 9)

    Layers = [l1, l2, l3, l4, l5, l6, l7, l8, l9]
    Layers = Layers[:n_layers]

    random_int = random.randint(0, 29)
    train = load_models_dict(path="../data/models/mnist/all/test_ensemble")[random_int]
    
    unlearned = gefeu._main(
        model=train,
        dataset_name="mnist",
        t_Epochs = opt_Epochs,
        t_Learning_Rate = opt_Learning_Rate,
        t_Batch_Size = opt_Batch_Size,
        t_N2R_Ratio= opt_N2R_Ratio,
        t_Regularization_term = opt_Regularization_term,
        t_Layers = Layers,
        t_Noise_Dim = opt_Noise_Dim,
        t_Impair_LR=opt_Impair_LR,
        t_Repair_LR=opt_Repair_LR,
        logs=False,
        model_eval_logs=False,
    )

    valid_ds = mlp_dataclass.MNIST_CostumDataset(
        sample_mode="all",
        train=False,
        test=True,
        balanced=False,
        dataset_name="mnist",
        download=False,
    )  
    valid_dl = DataLoader(valid_ds, 256, shuffle=False)

    random_int = random.randint(0, 29)
    exact = load_models_dict(path="../data/models/mnist/except_erased/test_ensemble")[random_int]
    
    div = metrics.kl_divergence_between_models(
        model1 = exact,
        model2 = unlearned,
        data_loader = valid_dl,
    )

    return div

study_gefeu.optimize(objective, n_trials=3)

import pickle

# Save the sampler with pickle to be loaded later.
with open("sampler_gefeu.pkl", "wb") as fout:
    pickle.dump(study_gefeu.sampler, fout)

[W 2025-01-09 09:43:17,396] Trial 15 failed with parameters: {'opt_Epochs': 5, 'opt_Learning_Rate': 0.1057274899858981, 'opt_Batch_Size': 72, 'opt_N2R_Ratio': 19.396307415554414, 'opt_Regularization_term': 0.20168989401505294, 'opt_Noise_Dim': 128, 'opt_Impair_LR': 0.23015044491715436, 'opt_Repair_LR': 0.141910032906416, 'l1': 368, 'l2': 1009, 'l3': 541, 'l4': 903, 'l5': 1007, 'l6': 318, 'l7': 710, 'l8': 990, 'l9': 195, 'n_layers': 2} because of the following error: The value nan is not acceptable.


Trial 15 failed with parameters: {'opt_Epochs': 5, 'opt_Learning_Rate': 0.1057274899858981, 'opt_Batch_Size': 72, 'opt_N2R_Ratio': 19.396307415554414, 'opt_Regularization_term': 0.20168989401505294, 'opt_Noise_Dim': 128, 'opt_Impair_LR': 0.23015044491715436, 'opt_Repair_LR': 0.141910032906416, 'l1': 368, 'l2': 1009, 'l3': 541, 'l4': 903, 'l5': 1007, 'l6': 318, 'l7': 710, 'l8': 990, 'l9': 195, 'n_layers': 2} because of the following error: The value nan is not acceptable.
Trial 15 failed with parameters: {'opt_Epochs': 5, 'opt_Learning_Rate': 0.1057274899858981, 'opt_Batch_Size': 72, 'opt_N2R_Ratio': 19.396307415554414, 'opt_Regularization_term': 0.20168989401505294, 'opt_Noise_Dim': 128, 'opt_Impair_LR': 0.23015044491715436, 'opt_Repair_LR': 0.141910032906416, 'l1': 368, 'l2': 1009, 'l3': 541, 'l4': 903, 'l5': 1007, 'l6': 318, 'l7': 710, 'l8': 990, 'l9': 195, 'n_layers': 2} because of the following error: The value nan is not acceptable.


[W 2025-01-09 09:43:17,398] Trial 15 failed with value nan.


Trial 15 failed with value nan.
Trial 15 failed with value nan.


[I 2025-01-09 09:54:37,764] Trial 16 finished with value: 1.2158881768584249 and parameters: {'opt_Epochs': 10, 'opt_Learning_Rate': 0.10894406201518292, 'opt_Batch_Size': 51, 'opt_N2R_Ratio': 18.578874381487278, 'opt_Regularization_term': 0.056790914585226886, 'opt_Noise_Dim': 355, 'opt_Impair_LR': 0.07639295637763363, 'opt_Repair_LR': 0.22081165973501782, 'l1': 424, 'l2': 284, 'l3': 841, 'l4': 832, 'l5': 803, 'l6': 336, 'l7': 366, 'l8': 472, 'l9': 406, 'n_layers': 5}. Best is trial 16 with value: 1.2158881768584249.


Trial 16 finished with value: 1.2158881768584249 and parameters: {'opt_Epochs': 10, 'opt_Learning_Rate': 0.10894406201518292, 'opt_Batch_Size': 51, 'opt_N2R_Ratio': 18.578874381487278, 'opt_Regularization_term': 0.056790914585226886, 'opt_Noise_Dim': 355, 'opt_Impair_LR': 0.07639295637763363, 'opt_Repair_LR': 0.22081165973501782, 'l1': 424, 'l2': 284, 'l3': 841, 'l4': 832, 'l5': 803, 'l6': 336, 'l7': 366, 'l8': 472, 'l9': 406, 'n_layers': 5}. Best is trial 16 with value: 1.2158881768584249.
Trial 16 finished with value: 1.2158881768584249 and parameters: {'opt_Epochs': 10, 'opt_Learning_Rate': 0.10894406201518292, 'opt_Batch_Size': 51, 'opt_N2R_Ratio': 18.578874381487278, 'opt_Regularization_term': 0.056790914585226886, 'opt_Noise_Dim': 355, 'opt_Impair_LR': 0.07639295637763363, 'opt_Repair_LR': 0.22081165973501782, 'l1': 424, 'l2': 284, 'l3': 841, 'l4': 832, 'l5': 803, 'l6': 336, 'l7': 366, 'l8': 472, 'l9': 406, 'n_layers': 5}. Best is trial 16 with value: 1.2158881768584249.


[W 2025-01-09 09:58:17,365] Trial 17 failed with parameters: {'opt_Epochs': 10, 'opt_Learning_Rate': 0.2194577995561068, 'opt_Batch_Size': 162, 'opt_N2R_Ratio': 7.143927641052668, 'opt_Regularization_term': 0.16047532993361482, 'opt_Noise_Dim': 33, 'opt_Impair_LR': 0.025846138686518624, 'opt_Repair_LR': 0.17186523349146535, 'l1': 654, 'l2': 416, 'l3': 271, 'l4': 685, 'l5': 129, 'l6': 69, 'l7': 979, 'l8': 1007, 'l9': 986, 'n_layers': 1} because of the following error: The value nan is not acceptable.


Trial 17 failed with parameters: {'opt_Epochs': 10, 'opt_Learning_Rate': 0.2194577995561068, 'opt_Batch_Size': 162, 'opt_N2R_Ratio': 7.143927641052668, 'opt_Regularization_term': 0.16047532993361482, 'opt_Noise_Dim': 33, 'opt_Impair_LR': 0.025846138686518624, 'opt_Repair_LR': 0.17186523349146535, 'l1': 654, 'l2': 416, 'l3': 271, 'l4': 685, 'l5': 129, 'l6': 69, 'l7': 979, 'l8': 1007, 'l9': 986, 'n_layers': 1} because of the following error: The value nan is not acceptable.
Trial 17 failed with parameters: {'opt_Epochs': 10, 'opt_Learning_Rate': 0.2194577995561068, 'opt_Batch_Size': 162, 'opt_N2R_Ratio': 7.143927641052668, 'opt_Regularization_term': 0.16047532993361482, 'opt_Noise_Dim': 33, 'opt_Impair_LR': 0.025846138686518624, 'opt_Repair_LR': 0.17186523349146535, 'l1': 654, 'l2': 416, 'l3': 271, 'l4': 685, 'l5': 129, 'l6': 69, 'l7': 979, 'l8': 1007, 'l9': 986, 'n_layers': 1} because of the following error: The value nan is not acceptable.


[W 2025-01-09 09:58:17,365] Trial 17 failed with value nan.


Trial 17 failed with value nan.
Trial 17 failed with value nan.


In [6]:
u = load_models_dict(path="../data/models/mnist/all/test_ensemble")[1]
z = load_models_dict(path="../data/models/mnist/except_erased/test_ensemble")[1]
valid_ds = mlp_dataclass.MNIST_CostumDataset(
        sample_mode="all",
        train=False,
        test=True,
        balanced=False,
        dataset_name="mnist",
        download=False,
    )  
valid_dl = DataLoader(valid_ds, 56, shuffle=False)

In [7]:
metrics.kl_divergence_between_models(u, z, valid_dl, device='cpu')

0.04689864512512139

## Standard Values

In [ ]:
# TODO be determind

# standard_model = src.gefeu._main(
#     model=None,
#     dataset_name="mnist",
#     t_Epochs=,
#     t_Batch_Size=,
#     t_Learning_Rate=,
#     t_N2R_Ratio=,
#     t_Regularization_term=,
#     t_Layers=,
#     t_Noise_Dim=,
#     t_Impair_LR=,
#     t_Repair_LR=,
#     logs=True,
#     model_eval_logs=False,
# )

___
## GEMU

In [ ]:
# Add stream handler of stdout to show the messages
optuna.logging.get_logger("optuna").addHandler(logging.StreamHandler(sys.stdout))
study2_name = "GenOptiGEMU"  # Unique identifier of the study.
storage2_name = "sqlite:///{}.db".format(study2_name)

if os.path.exists("sampler_gemu.pkl"):
    restored2_sampler = pickle.load(open("sampler_gemu.pkl", "rb"))
study_gemu = optuna.create_study(study_name=study2_name, storage=storage2_name, load_if_exists=True, sampler=restored2_sampler)

In [ ]:
def objective(trial):

    opt_Epochs = trial.suggest_int('opt_Epochs', 1, 10)
    opt_Learning_Rate = trial.suggest_float('opt_Learning_Rate', 0.01, 0.3)
    opt_Batch_Size = trial.suggest_int('opt_Batch_Size', 32, 256)
    opt_N2R_Ratio = trial.suggest_float('opt_N2R_Ratio', 0.01, 20)
    opt_Regularization_term = trial.suggest_float('opt_Regularization_term', 0.01, 0.3)
    opt_Noise_Dim = trial.suggest_int('opt_Noise_Dim', 1, 512)
    opt_Impair_LR = trial.suggest_float('opt_Impair_LR', 0.01, 0.3)
    opt_Repair_LR = trial.suggest_float('opt_Repair_LR', 0.01, 0.3)

    l1 = trial.suggest_int('l1', 32, 1024)
    l2 = trial.suggest_int('l2', 32, 1024)
    l3 = trial.suggest_int('l3', 32, 1024)
    l4 = trial.suggest_int('l4', 32, 1024)
    l5 = trial.suggest_int('l5', 32, 1024)
    l6 = trial.suggest_int('l6', 32, 1024)
    l7 = trial.suggest_int('l7', 32, 1024)
    l8 = trial.suggest_int('l8', 32, 1024)
    l9 = trial.suggest_int('l9', 32, 1024)
    n_layers = trial.suggest_int('n_layers', 1, 9)

    Layers = [l1, l2, l3, l4, l5, l6, l7, l8, l9]
    Layers = Layers[:n_layers]

    random_int = random.randint(0, 29)
    train = load_models_dict(path="../data/models/mnist/all/test_ensemble")[random_int]

    unlearned = gemu._main(
        model=train,
        dataset_name="mnist",
        t_Epochs = opt_Epochs,
        t_Learning_Rate = opt_Learning_Rate,
        t_Batch_Size = opt_Batch_Size,
        t_N2R_Ratio= opt_N2R_Ratio,
        t_Regularization_term = opt_Regularization_term,
        t_Layers = Layers,
        t_Noise_Dim = opt_Noise_Dim,
        t_Impair_LR=opt_Impair_LR,
        t_Repair_LR=opt_Repair_LR,
        logs=False,
        model_eval_logs=False,
    )

    valid_ds = mlp_dataclass.MNIST_CostumDataset(
        sample_mode="all",
        train=False,
        test=True,
        balanced=False,
        dataset_name="mnist",
        download=False,
    )  
    valid_dl = DataLoader(valid_ds, 256, shuffle=False)

    random_int = random.randint(0, 29)
    exact = load_models_dict(path="../data/models/mnist/except_erased/test_ensemble")[random_int]
    
    div = metrics.kl_divergence_between_models(
        model1 = exact,
        model2 = unlearned,
        data_loader = valid_dl,
    )

    return div

study_gemu.optimize(objective, n_trials=3)

import pickle

# Save the sampler with pickle to be loaded later.
with open("sampler_gemu.pkl", "wb") as fout:
    pickle.dump(study_gemu.sampler, fout)

## Standard Values

In [ ]:
# TODO be determind

# standard_model = src.gefeu._main(
#     model=None,
#     dataset_name="mnist",
#     t_Epochs=,
#     t_Batch_Size=,
#     t_Learning_Rate=,
#     t_N2R_Ratio=,
#     t_Regularization_term=,
#     t_Layers=,
#     t_Noise_Dim=,
#     t_Impair_LR=,
#     t_Repair_LR=,
#     logs=True,
#     model_eval_logs=False,
# )

___
## Gradient Ascent

In [ ]:
# Add stream handler of stdout to show the messages
optuna.logging.get_logger("optuna").addHandler(logging.StreamHandler(sys.stdout))
study3_name = "OptiGA"  # Unique identifier of the study.
storage3_name = "sqlite:///{}.db".format(study2_name)

if os.path.exists("sampler_ga.pkl"):
    restored3_sampler = pickle.load(open("sampler_ga.pkl", "rb"))
study_ga = optuna.create_study(study_name=study3_name, storage=storage3_name, load_if_exists=True, sampler=restored3_sampler)

In [ ]:
from unlearning import SimpleGradientAscent

def objective(trial):

    opt_Epochs = trial.suggest_int('opt_Epochs', 1, 10)
    opt_Learning_Rate = trial.suggest_float('opt_Learning_Rate', 0.01, 0.4)
    opt_Batch_Size = trial.suggest_int('opt_Batch_Size', 1, 256)

    random_int = random.randint(0, 29)
    train = load_models_dict(path="../data/models/mnist/all/test_ensemble")[random_int]

    forget_ds = mlp_dataclass.MNIST_CostumDataset(
        sample_mode="only_erased",
        train=True,
        test=False,
        balanced=False,
        dataset_name="mnist",
        download=False,
    )
    forget_dl = valid_dl = DataLoader(forget_ds, opt_Batch_Size, shuffle=False)

    unlearned = SimpleGradientAscent(
        model=train,
        unlearned_data=forget_dl,
        dataset_name="mnist",
        t_LR = opt_Learning_Rate,
        t_Epochs = opt_Epochs,
        )

    valid_ds = mlp_dataclass.MNIST_CostumDataset(
        sample_mode="all",
        train=False,
        test=True,
        balanced=False,
        dataset_name="mnist",
        download=False,
    )  
    valid_dl = DataLoader(valid_ds, 256, shuffle=False)

    random_int = random.randint(0, 29)
    exact = load_models_dict(path="../data/models/mnist/except_erased/test_ensemble")[random_int]
    
    div = metrics.kl_divergence_between_models(
        model1 = exact,
        model2 = unlearned,
        data_loader = valid_dl,
    )

    return div

study_gemu.optimize(objective, n_trials=3)

import pickle

# Save the sampler with pickle to be loaded later.
with open("sampler_gemu.pkl", "wb") as fout:
    pickle.dump(study_gemu.sampler, fout)

## Standard Values

In [ ]:
# TODO be determind

# forget_ds = mlp_dataclass.MNIST_CostumDataset(
#     sample_mode="only_erased",
#     train=True,
#     test=False,
#     balanced=False,
#     dataset_name="mnist",
#     download=False,
# )
# forget_dl = valid_dl = DataLoader(forget_ds, , shuffle=False)

# unlearned = SimpleGradientAscent(
#     model=train,
#     unlearned_data=forget_dl,
#     dataset_name="mnist",
#     t_LR =,
#     t_Epochs =,
#     )

___